In [1]:
!pip install ctgan
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.8/148.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [2]:
from ctgan import CTGAN

import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import time

# record start time
start = time.time()
le = LabelEncoder()

df = pd.read_csv('https://raw.githubusercontent.com/mahayasa/gan-hybrid-sampling-customer-churn/main/data/telco_new.csv')

batch_size = 5000
epochs = 100
model = CTGAN(batch_size=batch_size, epochs=epochs, verbose=True)
df1=df[df['Churn']==0]
df=df[df['Churn']==1]
model.fit(df)

end = time.time()



print("The time of execution of preprocess:",
      (end-start), "s")

Gen. (-0.94) | Discrim. (-0.10): 100%|██████████| 100/100 [01:13<00:00,  1.36it/s]

The time of execution of preprocess: 87.40281414985657 s


In [3]:
n_generated_data = 2160
generated_df = model.sample(n_generated_data)

#concat original data and gan data
data_concat = pd.concat([df, generated_df])
# combine data churn and not churn
data=pd.concat([df1, data_concat])

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import sklearn.metrics as mt
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
from sklearn.ensemble import RandomForestClassifier
import math
import time

startu=time.time()

X=data.drop(['Churn'],axis=1)
y=data["Churn"]


#enn = EditedNearestNeighbours(n_neighbors=3)
#X, y = enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)
#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)
cv = KFold(n_splits=5, random_state=1, shuffle=True)

endu=time.time()
print("The time of execution of preprocess undersampling:",
      (endu-startu), "s")


The time of execution of preprocess undersampling: 0.0023908615112304688 s


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-telco3-dt-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-telco3-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.959261 using {'class_weight': {0: 90, 1: 100}}
0.950444 (0.007734) with: {'class_weight': {0: 10, 1: 100}}
0.954662 (0.006331) with: {'class_weight': {0: 20, 1: 100}}
0.955941 (0.005935) with: {'class_weight': {0: 30, 1: 100}}
0.955364 (0.005195) with: {'class_weight': {0: 40, 1: 100}}
0.956899 (0.004233) with: {'class_weight': {0: 50, 1: 100}}
0.955653 (0.006577) with: {'class_weight': {0: 60, 1: 100}}
0.958149 (0.005301) with: {'class_weight': {0: 70, 1: 100}}
0.958760 (0.006319) with: {'class_weight': {0: 80, 1: 100}}
0.959261 (0.005808) with: {'class_weight': {0: 90, 1: 100}}
0.958799 (0.006111) with: {'class_weight': {0: 100, 1: 100}}


In [19]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-telco3-dt-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-telco3-dt-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.954926 using {'class_weight': {0: 80, 1: 100}}
0.944940 (0.007603) with: {'class_weight': {0: 10, 1: 100}}
0.950782 (0.005999) with: {'class_weight': {0: 20, 1: 100}}
0.951472 (0.005754) with: {'class_weight': {0: 30, 1: 100}}
0.951662 (0.005933) with: {'class_weight': {0: 40, 1: 100}}
0.953733 (0.005337) with: {'class_weight': {0: 50, 1: 100}}
0.952039 (0.006608) with: {'class_weight': {0: 60, 1: 100}}
0.953985 (0.006282) with: {'class_weight': {0: 70, 1: 100}}
0.954926 (0.006264) with: {'class_weight': {0: 80, 1: 100}}
0.954424 (0.006235) with: {'class_weight': {0: 90, 1: 100}}
0.953041 (0.007580) with: {'class_weight': {0: 100, 1: 100}}


In [18]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-telco3-dt-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-telco3-dt-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder


Best: 0.956104 using {'class_weight': {0: 80, 1: 100}}
0.944945 (0.007573) with: {'class_weight': {0: 10, 1: 100}}
0.949185 (0.006419) with: {'class_weight': {0: 20, 1: 100}}
0.951633 (0.005449) with: {'class_weight': {0: 30, 1: 100}}
0.951706 (0.006550) with: {'class_weight': {0: 40, 1: 100}}
0.953583 (0.005013) with: {'class_weight': {0: 50, 1: 100}}
0.952519 (0.006534) with: {'class_weight': {0: 60, 1: 100}}
0.953592 (0.004937) with: {'class_weight': {0: 70, 1: 100}}
0.956104 (0.006989) with: {'class_weight': {0: 80, 1: 100}}
0.953331 (0.006683) with: {'class_weight': {0: 90, 1: 100}}
0.952883 (0.007749) with: {'class_weight': {0: 100, 1: 100}}


In [20]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-telco3-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-telco3-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.851537 using {'class_weight': {0: 100, 1: 100}}
0.823735 (0.020502) with: {'class_weight': {0: 10, 1: 100}}
0.832019 (0.014659) with: {'class_weight': {0: 20, 1: 100}}
0.837817 (0.015394) with: {'class_weight': {0: 30, 1: 100}}
0.840672 (0.013702) with: {'class_weight': {0: 40, 1: 100}}
0.843894 (0.013121) with: {'class_weight': {0: 50, 1: 100}}
0.845644 (0.011838) with: {'class_weight': {0: 60, 1: 100}}
0.846061 (0.013161) with: {'class_weight': {0: 70, 1: 100}}
0.848147 (0.014079) with: {'class_weight': {0: 80, 1: 100}}
0.849130 (0.012663) with: {'class_weight': {0: 90, 1: 100}}
0.851537 (0.012304) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-telco3-lr-cs-f1.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-telco3-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.757902 using {'class_weight': {0: 80, 1: 100}}
0.680617 (0.017205) with: {'class_weight': {0: 10, 1: 100}}
0.709170 (0.015084) with: {'class_weight': {0: 20, 1: 100}}
0.716490 (0.015837) with: {'class_weight': {0: 30, 1: 100}}
0.717289 (0.016843) with: {'class_weight': {0: 40, 1: 100}}
0.725093 (0.017876) with: {'class_weight': {0: 50, 1: 100}}
0.738125 (0.015569) with: {'class_weight': {0: 60, 1: 100}}
0.751455 (0.013097) with: {'class_weight': {0: 70, 1: 100}}
0.757902 (0.015088) with: {'class_weight': {0: 80, 1: 100}}
0.757237 (0.012737) with: {'class_weight': {0: 90, 1: 100}}
0.757469 (0.013921) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-telco3-lr-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-telco3-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.755917 using {'class_weight': {0: 100, 1: 100}}
0.646270 (0.020124) with: {'class_weight': {0: 10, 1: 100}}
0.682275 (0.017394) with: {'class_weight': {0: 20, 1: 100}}
0.694604 (0.017347) with: {'class_weight': {0: 30, 1: 100}}
0.698195 (0.018025) with: {'class_weight': {0: 40, 1: 100}}
0.710193 (0.019250) with: {'class_weight': {0: 50, 1: 100}}
0.727227 (0.016921) with: {'class_weight': {0: 60, 1: 100}}
0.744385 (0.013861) with: {'class_weight': {0: 70, 1: 100}}
0.753304 (0.015526) with: {'class_weight': {0: 80, 1: 100}}
0.754425 (0.013164) with: {'class_weight': {0: 90, 1: 100}}
0.755917 (0.014238) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-telco3-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-telco3-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.905771 using {'class_weight': {0: 20, 1: 100}}
0.858639 (0.013273) with: {'class_weight': {0: 10, 1: 100}}
0.905771 (0.013326) with: {'class_weight': {0: 20, 1: 100}}
0.904863 (0.013439) with: {'class_weight': {0: 30, 1: 100}}
0.901597 (0.013295) with: {'class_weight': {0: 40, 1: 100}}
0.901549 (0.012541) with: {'class_weight': {0: 50, 1: 100}}
0.902745 (0.012834) with: {'class_weight': {0: 60, 1: 100}}
0.904013 (0.012996) with: {'class_weight': {0: 70, 1: 100}}
0.904198 (0.013227) with: {'class_weight': {0: 80, 1: 100}}
0.904312 (0.013094) with: {'class_weight': {0: 90, 1: 100}}
0.904878 (0.012933) with: {'class_weight': {0: 100, 1: 100}}


In [24]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-telco3-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/ctgan-telco3-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.798489 using {'class_weight': {0: 80, 1: 100}}
0.691094 (0.013225) with: {'class_weight': {0: 10, 1: 100}}
0.704560 (0.013082) with: {'class_weight': {0: 20, 1: 100}}
0.719872 (0.010994) with: {'class_weight': {0: 30, 1: 100}}
0.759618 (0.012806) with: {'class_weight': {0: 40, 1: 100}}
0.780512 (0.011464) with: {'class_weight': {0: 50, 1: 100}}
0.789504 (0.013566) with: {'class_weight': {0: 60, 1: 100}}
0.796110 (0.013535) with: {'class_weight': {0: 70, 1: 100}}
0.798489 (0.013791) with: {'class_weight': {0: 80, 1: 100}}
0.798384 (0.015382) with: {'class_weight': {0: 90, 1: 100}}
0.798014 (0.015782) with: {'class_weight': {0: 100, 1: 100}}


In [25]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-telco3-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/ctgan-telco3-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.789750 using {'class_weight': {0: 100, 1: 100}}
0.657539 (0.015790) with: {'class_weight': {0: 10, 1: 100}}
0.673736 (0.015482) with: {'class_weight': {0: 20, 1: 100}}
0.693911 (0.012871) with: {'class_weight': {0: 30, 1: 100}}
0.741071 (0.014307) with: {'class_weight': {0: 40, 1: 100}}
0.765592 (0.012456) with: {'class_weight': {0: 50, 1: 100}}
0.776599 (0.014456) with: {'class_weight': {0: 60, 1: 100}}
0.784355 (0.014417) with: {'class_weight': {0: 70, 1: 100}}
0.787770 (0.014668) with: {'class_weight': {0: 80, 1: 100}}
0.788896 (0.015932) with: {'class_weight': {0: 90, 1: 100}}
0.789750 (0.016218) with: {'class_weight': {0: 100, 1: 100}}
